<a href="https://colab.research.google.com/github/willburr/humorous-headlines/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Coursework coding instructions (please also see full coursework spec)

Please choose if you want to do either Task 1 or Task 2. You should write your report about one task only.

For the task you choose you will need to do two approaches:
  - Approach 1, which can use use pre-trained embeddings / models
  - Approach 2, which should not use any pre-trained embeddings or models
We should be able to run both approaches from the same colab file

#### Running your code:
  - Your models should run automatically when running your colab file without further intervention
  - For each task you should automatically output the performance of both models
  - Your code should automatically download any libraries required

#### Structure of your code:
  - You are expected to use the 'train', 'eval' and 'model_performance' functions, although you may edit these as required
  - Otherwise there are no restrictions on what you can do in your code

#### Documentation:
  - You are expected to produce a .README file summarising how you have approached both tasks

#### Reproducibility:
  - Your .README file should explain how to replicate the different experiments mentioned in your report

Good luck! We are really looking forward to seeing your reports and your model code!

# Project  Set-up

The code blocks below provide the imports and necessaryset-up steps to run later sections. We import necessary libraries, set pytorch to use the GPU and load and unzip the data.

In [ ]:
# Imports

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import Dataset, random_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import torch.optim as optim
import codecs
import tqdm

In [ ]:
# Setting random seed and device
SEED = 1

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

## Downloading and unzipping the data

TODO: Eventually remove drive import and replace with !wget (Might be faster to just do this anyway) 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Unzip data
!unzip "/content/drive/MyDrive/cw/task-2-20210202T162438Z-001.zip"

In [ ]:
# Load data
train_df = pd.read_csv('task-2/train.csv')
test_df = pd.read_csv('task-2/dev.csv')

In [ ]:
# Take a look at the data
train_df.head()

## Download and unzip GLOVE Embeddings

TODO: Do we bother using GLOVE at all? May be interesting in report to compare GLOVE vs BERT embeddings?

In [ ]:
# You will need to download any word embeddings required for your code, e.g.:

# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip

# For any packages that Colab does not provide auotmatically you will also need to install these below, e.g.:

#! pip install torch

In [ ]:
!unzip '/content/drive/MyDrive/cw/glove.6B.zip'

## Download and install Transformers library

TODO: Will move BERT install stuff here (and imports to top)

## Training Evaluation Utility Functions

In [ ]:
# We define our training loop
def train(train_iter, dev_iter, model, number_epoch):
    """
    Training loop for the model, which calls on eval to evaluate after each epoch
    """

    print("Training model.")

    for epoch in range(1, number_epoch+1):
        
        model.train()
        
        epoch_loss = 0
        epoch_correct = 0
        no_observations = 0  # Observations used for training so far

        for batch in train_iter:
            feature, target = batch
            feature, target = feature.to(device), target.to(device)

            # for RNN:
            model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            model.hidden = model.init_hidden()

            predictions = model(feature).squeeze(1)
            optimizer.zero_grad()
            loss = loss_fn(predictions, target)

            correct, __ = model_performance(np.argmax(predictions.detach().cpu().numpy(), axis=1), target.detach().cpu().numpy())

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()*target.shape[0]
            epoch_correct += correct

        valid_loss, valid_acc, __, __ = eval(dev_iter, model)

        epoch_loss, epoch_acc = epoch_loss / no_observations, epoch_correct / no_observations
        print(f'| Epoch: {epoch:02} | Train Loss: {epoch_loss:.2f} | Train Accuracy: {epoch_acc:.2f} | \
        Val. Loss: {valid_loss:.2f} | Val. Accuracy: {valid_acc:.2f} |')

In [ ]:
# We evaluate performance on our dev set
def eval(data_iter, model):
    """
    Evaluating model performance on the dev set
    """
    model.eval()
    epoch_loss = 0
    epoch_correct = 0
    pred_all = []
    trg_all = []
    no_observations = 0

    with torch.no_grad():
        for batch in data_iter:
            feature, target = batch

            feature, target = feature.to(device), target.to(device)

            # for RNN:
            model.batch_size = target.shape[0]
            no_observations = no_observations + target.shape[0]
            model.hidden = model.init_hidden()

            predictions = model(feature).squeeze(1)
            loss = loss_fn(predictions, target)

            # We get the mse
            pred, trg = predictions.detach().cpu().numpy(), target.detach().cpu().numpy()
            correct, __ = model_performance(np.argmax(pred, axis=1), trg)

            epoch_loss += loss.item()*target.shape[0]
            epoch_correct += correct
            pred_all.extend(pred)
            trg_all.extend(trg)

    return epoch_loss/no_observations, epoch_correct/no_observations, np.array(pred_all), np.array(trg_all)

In [ ]:
# How we print the model performance
def model_performance(output, target, print_output=False):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    correct_answers = (output == target)
    correct = sum(correct_answers)
    acc = np.true_divide(correct,len(output))

    if print_output:
        print(f'| Acc: {acc:.2f} ')

    return correct, acc

# Data augmentation and Vocab Preparation

## Data augmentation

In [ ]:
import re

def substitute(sentence, edit):
  open_pos = sentence.find('<')
  close_pos = sentence.find('>')
  sub = sentence.replace(sentence[open_pos: close_pos + 1], edit)
  return sub

def get_edited_df(df):
  edited1 = df.apply(lambda x:substitute(x['original1'], x['edit1']), axis=1)
  edited2 = df.apply(lambda x:substitute(x['original2'], x['edit2']), axis=1)
  combined = list(zip(edited1, edited2))
  return pd.DataFrame(combined, columns=['edited1', 'edited2'])

edited_train_df = get_edited_df(train_df)
edited_test_df = get_edited_df(test_df)

edited_train_df.head()

## Vocab Preparation

In [ ]:
# To create our vocab
def create_vocab(data):
    """
    Creating a corpus of all the tokens used
    """
    tokenized_corpus = [] # Let us put the tokenized corpus in a list

    for sentence_pair in data:
        tokenized_sentence_pair = []
        for sentence in sentence_pair:
            tokenized_sentence = []

            for token in sentence.split(' '): # simplest split is

                tokenized_sentence.append(token.lower())

            tokenized_sentence_pair.append(tokenized_sentence)
        tokenized_corpus.append(tokenized_sentence_pair)

    # Create single list of all vocabulary
    vocabulary = []  # Let us put all the tokens (mostly words) appearing in the vocabulary in a list

    for sentence_pair in tokenized_corpus:

        for sentence in sentence_pair:

            for token in sentence:

                if token.lower() not in vocabulary:

                    vocabulary.append(token.lower())

    return vocabulary, tokenized_corpus

TODO: Potentiallybreak up code blocks a bit more?

In [ ]:
## Approach 1 code, using functions defined above:

# We set our training data and test data
training_data = edited_train_df.values.tolist()
test_data = edited_test_df.values.tolist()

# Creating word vectors
training_vocab, training_tokenized_corpus = create_vocab(training_data)
test_vocab, test_tokenized_corpus = create_vocab(test_data)

# Creating joint vocab from test and train
# TODO: We can optimise to just use the previous two:
joint_vocab, joint_tokenized_corpus = create_vocab(training_data + test_data)

print("Vocab created.")

The below code block loads in the GLOVE embeddings.
TODO: We can move this into the Vocabulary class

In [ ]:
# We create representations for our tokens
wvecs = [] # word vectors
word2idx = [] # word2index
idx2word = []

# This is a large file, it will take a while to load in the memory!
with codecs.open('glove.6B.300d.txt', 'r','utf-8') as f:
  index = 1
  for line in f.readlines():
    # Ignore the first line - first line typically contains vocab, dimensionality
    if len(line.strip().split()) > 3:
      word = line.strip().split()[0]
      if word in joint_vocab:
          (word, vec) = (word,
                     list(map(float,line.strip().split()[1:])))
          wvecs.append(vec)
          word2idx.append((word, index))
          idx2word.append((index, word))
          index += 1


wvecs = np.array(wvecs)
word2idx = dict(word2idx)
idx2word = dict(idx2word)

vectorized_seqs = [[[word2idx[tok] for tok in sen if tok in word2idx] for sen in seq] for seq in training_tokenized_corpus]


## Load the Dataset

### Utility functions

In [ ]:
# Used for collating our observations into minibatches:
def collate_fn_padd(batch):
    '''
    We add padding to our minibatches and create tensors for our model
    '''
    
    batch_labels = [l for f, l in batch]
    batch_features = [f for f, l in batch]

    batch_features_len = [[len(s) for s in f] for f, l in batch]

    seq_tensor = torch.zeros((2, len(batch), np.max(batch_features_len))).long()

    # Shape of seq_tensor is batch_size x max_feature_len
    # It should be batch_size x 2 x max_feature_len

    for idx, (seq, seqlens) in enumerate(zip(batch_features, batch_features_len)):
        for i in range(2):
            seq_tensor[i, idx, :seqlens[i]] = torch.LongTensor(seq[i])

    batch_labels = torch.LongTensor(batch_labels)

    return seq_tensor, batch_labels

def flip_label(label):
  if label == 1:
    return 2
  elif label == 2:
    return 1
  else: 
    return label

# We create a Dataset so we can create minibatches
class Task2Dataset(Dataset):

    def __init__(self, train_data, labels):
        self.x_train = train_data
        self.y_train = labels

    def __len__(self):
        return len(self.y_train)

    def __getitem__(self, item):
        return self.x_train[item], self.y_train[item]

### Creating Dataloaders

In [ ]:
BATCH_SIZE = 32
train_proportion = 0.8

feature = vectorized_seqs

labels = list(train_df['label'])


# 'feature' is a list of lists, each containing embedding IDs for word tokens
train_and_dev = Task2Dataset(feature, labels)

train_examples = round(len(train_and_dev)*train_proportion)
dev_examples = len(train_and_dev) - train_examples

train_dataset, dev_dataset = random_split(train_and_dev,
                                           (train_examples,
                                            dev_examples))

# Add the reverse examples to the training set to create more training dara
flipped_train_dataset = []
for r in train_dataset:
  flipped_train_dataset.append((list(reversed(r[0])), flip_label(r[1])))

train_dataset += flipped_train_dataset

train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)
print("Dataloaders created.")

# Approach 1: Using pre-trained representations

## Using pre-trained embeddings with a BiLSTM model

In [ ]:
class BiLSTM_classification(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, batch_size, device):
        super(BiLSTM_classification, self).__init__()
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.device = device
        self.batch_size = batch_size
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2label = nn.Linear(hidden_dim * 4, 3)
        # self.hidden1 = self.init_hidden()
        # self.hidden2 = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly why they have this dimensionality.
        # The axes semantics are (num_layers * num_directions, minibatch_size, hidden_dim)
        return torch.zeros(2, self.batch_size, self.hidden_dim).to(self.device), \
               torch.zeros(2, self.batch_size, self.hidden_dim).to(self.device)

    def forward(self, sentence):
        try: 
          embedding1 = self.embedding(sentence[0])
        except IndexError: 
          print(sentence[0].shape)
          print(sentence[0])
        embedding1 = embedding1.permute(1, 0, 2)

        try: 
          embedding2 = self.embedding(sentence[1])
        except IndexError: 
          print(sentence[1].shape)
          print(sentence[1])
        embedding2 = embedding2.permute(1, 0, 2)

        lstm_out_1, _ = self.lstm(embedding1.view(len(embedding1), self.batch_size, self.embedding_dim))
        lstm_out_2, _ = self.lstm(embedding2.view(len(embedding2), self.batch_size, self.embedding_dim))
        # concat both lstm_out_1 and lstm_out_2 and give to our linear layer, think we need
        # output shape of lstm_ out is 2 * hidden_size as we do it in both directions
 
        # lstm_out_1[-1] are both batch_size x (2 * hidden_size)
        # do we concat so it's (2*batch_size) x (2 * hidden_size) or batch_size x (2*(2 * hidden_size))? - I chose the latter
        out = self.hidden2label(torch.cat((lstm_out_1[-1], lstm_out_2[-1]), 1))
        return out

TODO: Separate out dataset stuff and model stuff below and put data loading stuff in section above. 

In [ ]:
# Number of epochs
epochs = 16

INPUT_DIM = len(word2idx) + 1
HIDDEN_DIM = 400 # Higher appears to yield better val acc
EMBEDDING_DIM = 300 # Higher yields better val acc
BATCH_SIZE = 32

## TODO: Figure out network dimensions
model = BiLSTM_classification(EMBEDDING_DIM, HIDDEN_DIM, INPUT_DIM, BATCH_SIZE, device)
print("Model initialised.")

model.to(device)
# We provide the model with our embeddings
model.embedding.weight.data[1:].copy_(torch.from_numpy(wvecs))


In [ ]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

optimizer = torch.optim.Adam(model.parameters())

train(train_loader, dev_loader, model, epochs)

## Fine-tuning BERT

The following code is basically just an initial attempt to fine-tune BERT for the task. 
A good amount of the code here is either inspired by or, in the case of the training loop, literally taken from this link: 
https://github.com/aniruddhachoudhury/BERT-Tutorials/blob/master/Blog%202/BERT_Fine_Tuning_Sentence_Classification.ipynb

I know it needs a refactor. I just want it to work. 

I think we could also try to take some inspiration from the following two resources: 

- https://medium.com/@theoliao1998/humor-detection-with-a-bert-regressor-9dc0a821c294
- https://arxiv.org/pdf/2004.12765.pdf
- https://github.com/ceshine/pytorch-pretrained-BERT/blob/master/notebooks/Next%20Sentence%20Prediction.ipynb

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification 

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', 
    num_labels = 3,
    output_attentions = False,
    output_hidden_states = False)

model = model.to(device)

In [ ]:
# We set our training data and test data
training_data = edited_train_df.values.tolist()
test_data = edited_test_df.values.tolist()

training_labels = train_df['label'].values.tolist()

In [ ]:
input_ids = []

# For every sentence pair, we create a single combined sentence with a separator token
# After this loop, each pair becomes:
#    [CLS] + encode(sentence1) + [SEP] + encode(sentence2) + [SEP]
# CLS is a special token used by BERT for classification tasks
# SEP is a special token used for separating sentences (I'm not sure if we need the one at the end)

for (sentence1, sentence2) in training_data:
    encoded_sentence1 = tokenizer.encode(sentence1)
    encoded_sentence2 = tokenizer.encode(sentence2 + '[SEP]', add_special_tokens=False)
    
    # Add the encoded sentence to the list.
    input_ids.append(encoded_sentence1 + encoded_sentence2)

In [ ]:
from keras.preprocessing.sequence import pad_sequences

# Set the maximum sequence length.
# I've chosen 74 somewhat arbitrarily. It's slightly larger than the
# maximum training sentence length of 72...
MAX_LEN = 74

print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# Pad our input tokens with value 0.
# "post" indicates that we want to pad and truncate at the end of the sequence,
# as opposed to the beginning.
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
                          value=0, truncating="post", padding="post")

print('\nDone.')
attention_masks = []

# For each sentence...
for sent in input_ids:
    
    # Create the attention mask.
    #   - If a token ID is 0, then it's padding, set the mask to 0.
    #   - If a token ID is > 0, then it's a real token, set the mask to 1.
    att_mask = [int(token_id > 0) for token_id in sent]
    
    # Store the attention mask for this sentence.
    attention_masks.append(att_mask)

In [ ]:
# Used for collating our observations into minibatches:

# Note that this one adds an attention mask (used to tell BERT to ignore padding)
def collate_fn_padd_mask(batch):
    '''
    We add padding to our minibatches and create tensors for our model
    '''
    batch_labels = [l for f, l in batch]
    batch_features = [f for f, l in batch]

    batch_features_len = [len(f) for f, l in batch]

    seq_tensor = torch.zeros((len(batch), max(batch_features_len))).long()
    mask_tensor = torch.zeros((len(batch), max(batch_features_len))).long()

    for idx, (seq, seqlen) in enumerate(zip(batch_features, batch_features_len)):
        seq_tensor[idx, :seqlen] = torch.LongTensor(seq)
        mask_tensor[idx, :seqlen] = torch.ones(seqlen)

    batch_labels = torch.LongTensor(batch_labels)

    return seq_tensor, mask_tensor, batch_labels

In [ ]:
train_and_dev = Task2Dataset(input_ids, training_labels)

train_examples = round(len(train_and_dev)*train_proportion)
dev_examples = len(train_and_dev) - train_examples

train_dataset, dev_dataset = random_split(train_and_dev,
                                           (train_examples,
                                            dev_examples))

BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd_mask)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd_mask)
print("Dataloaders created.")

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),
                  lr = 1e-5,
                  eps = 1e-8
                )
from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in dev_loader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))

print("")
print("Training complete!")

# Approach 2: No pre-trained representations

## Define Vocab Class

TODO: probs move this up to top and have functions for using GLOVE, BERT and our own

In [ ]:
class Vocabulary(object):
  """Data structure representing the vocabulary of a corpus."""
  def __init__(self):
    # Mapping from tokens to integers
    self._word2idx = {}

    # Reverse-mapping from integers to tokens
    self.idx2word = []

    # 0-padding token
    self.add_word('<pad>')
    # sentence start
    self.add_word('<s>')
    # sentence end
    self.add_word('</s>')
    # Unknown words
    self.add_word('<unk>')

    self._unk_idx = self._word2idx['<unk>']

  def word2idx(self, word):
    """Returns the integer ID of the word or <unk> if not found."""
    return self._word2idx.get(word, self._unk_idx)

  def add_word(self, word):
    """Adds the `word` into the vocabulary."""
    if word not in self._word2idx:
      self.idx2word.append(word)
      self._word2idx[word] = len(self.idx2word) - 1

  def build_from_list(self, words):
    for word in words:
      self.add_word(word)

  def build_from_file(self, fname):
    """Builds a vocabulary from a given corpus file."""
    with open(fname) as f:
      for line in f:
        words = line.strip().split()
        for word in words:
          self.add_word(word)

  def convert_idxs_to_words(self, idxs):
    """Converts a list of indices to words."""
    return ' '.join(self.idx2word[idx] for idx in idxs)

  def convert_words_to_idxs(self, words):
    """Converts a list of words to a list of indices."""
    return [self.word2idx(w) for w in words]

  def __len__(self):
    """Returns the size of the vocabulary."""
    return len(self.idx2word)
  
  def __repr__(self):
    return "Vocabulary with {} items".format(self.__len__())

TODO: Make code work with sentence pairs
TODO: Can probably make below function part of Vocabulary()

In [ ]:
# def corpus_to_tensor(_vocab, corpus):
#   # Final token indices
#   idxs = []
  
#   for sent in corpus:
#         sent = f"<s> {line} </s>"
#         # Split from whitespace and add sentence markers
#         idxs.extend(_vocab.convert_words_to_idxs(line.split()))
#   return torch.LongTensor(idxs)

## Create word-index mappings

In [ ]:
vocab = Vocabulary()
vocab.build_from_list(joint_vocab)
print(vocab)
print(vocab.convert_words_to_idxs('the dancer is on the moon'.split()))

## Prepare data
TODO: Reduce duplication

In [ ]:
vectorized_seqs = [[[vocab._word2idx[tok] for tok in sen if tok in vocab._word2idx] for sen in seq] for seq in training_tokenized_corpus]

In [ ]:
BATCH_SIZE = 32
train_proportion = 0.8

feature = vectorized_seqs

labels = list(train_df['label']) + [(3.5*x)-(1.5*(x**2)) for x in train_df['label']]

# 'feature' is a list of lists, each containing embedding IDs for word tokens
train_and_dev = Task2Dataset(feature, labels)

train_examples = round(len(train_and_dev)*train_proportion)
dev_examples = len(train_and_dev) - train_examples

train_dataset, dev_dataset = random_split(train_and_dev,
                                           (train_examples,
                                            dev_examples))

train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=True, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=BATCH_SIZE, collate_fn=collate_fn_padd)
print("Dataloaders created.")

## Bi-LSTM model

In [ ]:
# Number of epochs
epochs = 16

INPUT_DIM = len(vocab._word2idx) + 1
HIDDEN_DIM = 400 # Higher appears to yield better val acc
EMBEDDING_DIM = 300 # Higher yields better val acc
BATCH_SIZE = 32

## TODO: Figure out network dimensions
model = BiLSTM_classification(EMBEDDING_DIM, HIDDEN_DIM, INPUT_DIM, BATCH_SIZE, device)
print("Model initialised.")
model.to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
loss_fn = loss_fn.to(device)

optimizer = torch.optim.Adam(model.parameters())

train(train_loader, dev_loader, model, epochs)

## LSTM model with attention

Below code was inspired by the following:


*   https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
*   https://medium.com/intel-student-ambassadors/implementing-attention-models-in-pytorch-f947034b3e66
* https://www.kaggle.com/yshubham/simple-lstm-for-text-classification-with-attention
* https://iopscience.iop.org/article/10.1088/1742-6596/1207/1/012008






In [ ]:
class AttnLSTM(nn.Module):
  
  def __init__(self, embedding_dim, hidden_dim, output_dim, vocab_size, dropout_prop=0.1):
    super(AttnLSTM, self).__init__()
    
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dimm
    self.output_dim = output_dim
    self.dropout_prop = dropout_prop
    
    self.embedding = nn.Embedding(
      num_embeddings=self.vocab_size, embedding_dim=self.embedding_dim,
      padding_idx=0)
    self.attn = nn.Linear(embedding_dim + hidden_dim, 1)
    self.dropout = nn.Dropout(self.dropout_prop)
    self.hidden = self.init_hidden()

    self.lstm = nn.LSTM(hidden_dim + vocab_size, output_size)
    self.final = nn.Linear(hideen_dim, output_dim)
  
  def init_hidden(self):
    return (torch.zeros(1, 1, self.output_size),
      torch.zeros(1, 1, self.output_size))
  
  def forward(self, input, prev_out):
    embedded = self.embedding(input).view(1, 1, -1)
    embedded = self.dropout(embedded)

    # Use MLP attention
    attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], self.hidden[0]), 1)), dim=1)
    
    # TODO: How tf does this work?
    attn_applied = torch.bmm(attn_weights.unsqueeze(0), 
                             prev_out.view(1, -1, self.hidden_dim))
    
    lstm_in = torch.cat((attn_applied, embedded), dim = 1)
    
    output, self.hidden = self.lstm(lstm_in.unsqueeze(0), self.hidden)
    
    output = self.final(output)

    ## TODO: What about softmax / sigmoid?
    # output = F.log_softmax(self.out(output[0]), dim=1)
    
    return output

## Training BERT from scratch

following code adapted from:
https://huggingface.co/blog/how-to-train

Tokenizer docs: https://github.com/huggingface/tokenizers/tree/master/bindings/pythonhttps://github.com/huggingface/tokenizers/tree/master/bindings/python


In [ ]:
# TODO: Move to top
from tokenizers import ByteLevelBPETokenizer

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# TODO: tokenize text with filepath? pass in train.csv? convert to txt?

# Customize training
tokenizer.train(files=paths, vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

# produces a vocab.json and a merges.txt

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification()

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

## Pre-provided code for approach 2:

In [ ]:
train_and_dev = train_df['edit1']

training_data, dev_data, training_y, dev_y = train_test_split(train_df['edit1'], train_df['label'],
                                                                        test_size=(1-train_proportion),
                                                                        random_state=42)

# We train a Tf-idf model
count_vect = CountVectorizer(stop_words='english')
train_counts = count_vect.fit_transform(training_data)
transformer = TfidfTransformer().fit(train_counts)
train_counts = transformer.transform(train_counts)
naive_model = MultinomialNB().fit(train_counts, training_y)

# Train predictions
predicted_train = naive_model.predict(train_counts)

# Calculate Tf-idf using train and dev, and validate on dev:
test_and_test_counts = count_vect.transform(train_and_dev)
transformer = TfidfTransformer().fit(test_and_test_counts)

test_counts = count_vect.transform(dev_data)

test_counts = transformer.transform(test_counts)

# Dev predictions
predicted = naive_model.predict(test_counts)

# We run the evaluation:
print("\nTrain performance:")

sse, mse = model_performance(predicted_train, training_y, True)

print("\nDev performance:")
sse, mse = model_performance(predicted, dev_y, True)

#### Baseline for task 2

In [ ]:
# Baseline for the task
pred_baseline = torch.zeros(len(dev_y)) + 1  # 1 is most common class
print("\nBaseline performance:")
sse, mse = model_performance(pred_baseline, torch.tensor(dev_y.values), True)